<a href="https://colab.research.google.com/github/Daniel-Brett-Stark/PortfolioProjects/blob/main/PDSLassoPortfolioProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import pandas as pd
import numpy as np
from sklearn.linear_model import LassoCV
from sklearn import linear_model
from sklearn.model_selection import KFold

Mounted at /content/gdrive


In [ ]:
#import df
df=pd.read_csv('/content/gdrive/My Drive/Econ 484/484Project/MergedData/AllCountriesMergedDtypes.csv')
print(df.head)

<bound method NDFrame.head of       Unnamed: 0    value1     value2      value3  value4    value5  value6  \
0              0  153.9510  15.449556   55.813258  12.400  0.037055    5.30   
1              1  150.1920  17.990082  170.825030  79.478  0.037488   32.64   
2              2  146.4330  38.627892   43.015907  35.840  0.048671   18.28   
3              3  143.7648  37.418855   31.488022  14.300  0.057008   10.30   
4              4  141.0966  29.721067   37.386922  42.080  0.045700   24.34   
...          ...       ...        ...         ...     ...       ...     ...   
1492        1492  104.3014   7.144479   24.841702  61.580  0.925816   33.18   
1493        1493   99.7598   8.745304   24.841702  80.000  0.894256   67.00   
1494        1494   86.1350   8.340969   27.234910  68.240  0.718570   33.08   
1495        1495   83.2486  10.520955   26.046247  77.660  0.849793   40.18   
1496        1496   80.3622  10.143657   13.209815  86.000  0.099858   33.08   

         value7      

In [ ]:
y = df.loc[:,'success']
d = df.loc[:,'value55']
X = df.drop(columns=['success', 'value55','countrycode','Unnamed: 0'],axis=1)

#step 1
lassoy = LassoCV(normalize=True).fit(X, y)

#step 2
lassod = LassoCV(normalize=True).fit(X, d)
print(lassod.coef_)

lassod = LassoCV(normalize=True).fit(X, d)

#step 3
Xunion=X.iloc[:,(lassod.coef_!=0) + (lassoy.coef_!=0)]
Xunion.head()

rhs=pd.concat([d,Xunion],axis=1)
fullreg=linear_model.LinearRegression().fit(rhs,y)
print("PDS regression effect of selective college: {:.3f}".format(fullreg.coef_[0]))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_base.py:145: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lasso())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_base.py:145: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from skle

[ 0.          0.          0.         -0.          0.         -0.
 -0.         -0.         -0.          0.          0.          0.
 -0.         -0.         -0.          0.         -0.         -0.
 -0.          0.         -0.         -0.         -0.          0.
  0.          0.          0.         -0.          0.          0.
 -0.          0.          0.         -0.          0.         -0.
  0.          0.          0.          0.          0.02939885  0.
  0.         -0.         -0.         -0.         -0.         -0.01090299
 -0.          0.         -0.          0.          0.         -0.
  0.         -0.          0.        ]
PDS regression effect of selective college: 0.150


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_base.py:145: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lasso())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  FutureWarning,


In [ ]:
print("PDS regression effect of unemployment on terrorist attacks: {:.3f}".format(fullreg.coef_[0]))

PDS regression effect of unemployment on terrorist attacks: 0.150


In [ ]:
# create our sample splitting "object"
kf = KFold(n_splits=5,shuffle=True,random_state=42)

# apply the splits to our Xs
kf.get_n_splits(X)

# initialize array to hold each fold's regression coefficient
coeffs=np.zeros(5)

# Now loop through each fold
ii=0
for train_index, test_index in kf.split(X):
  X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
  y_train, y_test = y.iloc[train_index], y.iloc[test_index]
  d_train, d_test = d.iloc[train_index], d.iloc[test_index]


  # Do DDML thing
  # Ridge y on training folds:
  lassoy.fit(X_train, y_train)

  # but get residuals in test set
  yresid=y_test-lassoy.predict(X_test)
  
  #Ridge d on training folds
  lassod.fit(X_train,d_train)

  #but get residuals in test set
  dresid=d_test-lassod.predict(X_test)
  dresid=np.array(dresid).reshape(-1,1)
  
  # regress resids on resids
  ddmlreg=linear_model.LinearRegression().fit(dresid,yresid)

  # save coefficient in a vector
  coeffs[ii]=ddmlreg.coef_[0]
  ii+=1




/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_base.py:145: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lasso())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_base.py:145: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from skle

In [ ]:

# Take average
print("Double-Debiased Machine Learning effect of unemployment on terrorism: {:.3f}".format(np.mean(coeffs)))
coeffs

Double-Debiased Machine Learning effect of recessions on terrorism: 0.268


array([-0.79826372, -0.12834875,  0.69493336,  0.19066353,  1.3796542 ])